In [1]:
import requests
import re
import pandas as pd

In [4]:
text = "X-rays were negative and physical assessment determined soft tissue damage to the lateral aspect of her ankle. She was initially treated with ice, an ace wrap, crutches and mild pain medications (Tylenol with codeine)"
text2 = "It is a skin disease causing much itchiness. Scratching leads to redness, swelling, cracking, weeping clear fluid, crusting, and scaling."
# out=query_raw(text)
# print(out)
# print(term_extraction(text,out,'drug'))
# print(term_extraction(text,out,'disease'))
# print(term_extraction(text,out,'species'))

### Query Function:
>This function will send our text to bern2 using their API to get the text labeled

In [164]:
def query_plain(text, url="http://bern2.korea.ac.kr/plain"):
    return requests.post(url, json={'text': text}).json()

if __name__ == '__main__':
    text = "Autophagy maintains tumour growth through circulating the great arginine."
    print(query_plain(text))

{'annotations': [{'id': ['mesh:D009369'], 'is_neural_normalized': False, 'mention': 'tumour', 'obj': 'disease', 'prob': 0.9999945163726807, 'span': {'begin': 20, 'end': 26}}, {'id': ['mesh:D001120'], 'is_neural_normalized': False, 'mention': 'arginine', 'obj': 'drug', 'prob': 0.9939508438110352, 'span': {'begin': 64, 'end': 72}}], 'text': 'Autophagy maintains tumour growth through circulating the great arginine.', 'timestamp': 'Fri Nov 11 19:00:41 +0000 2022'}


In [265]:
text = "Autophagy maintains tumour growth through circulating the great arginine."
text1 = "X-rays were negative and physical assessment determined soft tissue damage to the lateral aspect of her ankle. She was initially treated with ice, an ace wrap, crutches and mild pain medications (Tylenol with codeine)"
text2 = "It is a skin disease causing much itchiness. Scratching leads to redness, swelling, cracking, weeping clear fluid, crusting, and scaling."
texts = [text,text1,text2]

In [295]:
coms = pd.DataFrame({'comments':texts})
coms.reset_index(inplace=True)
coms.rename(columns = {"index":"idx"},inplace=True)
coms.idx = coms.idx.astype(str)
coms['comidx'] = coms.idx +': '+coms.comments+'. ::'
coms.comidx = coms.comidx.str.lower().str.replace(r'\(|\)',',',regex=True)
coms

,idx,comments,comidx
0,0,Autophagy maintains tumour growth through circ...,0: autophagy maintains tumour growth through c...
1,1,X-rays were negative and physical assessment d...,1: x-rays were negative and physical assessmen...
2,2,It is a skin disease causing much itchiness. S...,2: it is a skin disease causing much itchiness...


In [296]:
texty = ' '.join(coms.comidx.tolist())

In [13]:
# texty = ''
# text = 'Autophagy maintains tumour growth through circulating arginine'

# for o in range(2):

#     texty= texty+f'{o}: '+text+f' :{o} '

In [13]:
# texty = ''
# text = 'Autophagy maintains tumour growth through circulating arginine'

# for o in range(2):

#     texty= texty+f'{o}: '+text+f' :{o} '

In [ ]:
def query_plain(text, url="http://bern2.korea.ac.kr/plain"):
    return requests.post(url, json={'text': text}).json()

if __name__ == '__main__':
#     text = "Autophagy maintains tumour growth through circulating arginine"
#     print(query_plain(text))

#### Get Start and end index of each entry

In [297]:
texty

'0: autophagy maintains tumour growth through circulating the great arginine.. :: 1: x-rays were negative and physical assessment determined soft tissue damage to the lateral aspect of her ankle. she was initially treated with ice, an ace wrap, crutches and mild pain medications ,tylenol with codeine,. :: 2: it is a skin disease causing much itchiness. scratching leads to redness, swelling, cracking, weeping clear fluid, crusting, and scaling.. ::'

In [301]:
string = texty
pattern = r'(\d:.*?)::'
dfi = pd.DataFrame()
for o in re.findall(pattern,string):
        print(o)
        print()
        match=(re.search(o, string))
        print(match)
                #Getting the start and end index in tuple format using match.span()
        text = o
        start = match.start()
        end = match.end()
        span = match.span()
        dfi = dfi.append({'text':text,'start':start,'end':end,'span':span},ignore_index=True)
        print ("start and end index", match.span())
dfi   

0: autophagy maintains tumour growth through circulating the great arginine.. 

<re.Match object; span=(0, 78), match='0: autophagy maintains tumour growth through circ>
start and end index (0, 78)
1: x-rays were negative and physical assessment determined soft tissue damage to the lateral aspect of her ankle. she was initially treated with ice, an ace wrap, crutches and mild pain medications ,tylenol with codeine,. 

<re.Match object; span=(81, 303), match='1: x-rays were negative and physical assessment d>
start and end index (81, 303)
2: it is a skin disease causing much itchiness. scratching leads to redness, swelling, cracking, weeping clear fluid, crusting, and scaling.. 

<re.Match object; span=(306, 448), match='2: it is a skin disease causing much itchiness. s>
start and end index (306, 448)


,text,start,end,span
0,0: autophagy maintains tumour growth through c...,0.0,78.0,"(0, 78)"
1,1: x-rays were negative and physical assessmen...,81.0,303.0,"(81, 303)"
2,2: it is a skin disease causing much itchiness...,306.0,448.0,"(306, 448)"


# Send combined text to bern2

In [ ]:
output = query_plain(texty)

In [ ]:
# [ print(o) for o in output]

In [ ]:
# output['annotations'][0]


In [ ]:
# for o in output['annotations'][0]:
#     print(o)

In [ ]:
x = output

In [ ]:
dfa = pd.DataFrame(x['annotations'])

In [ ]:
dfa.columns

In [ ]:
dfa

In [81]:
# dfa['endsp']  = dfa[dfa.span['end']]

dfa.span = dfa.span.astype(str)
dfa ['str_end'] = dfa.span.str.replace(r".*'end': (\d+)}",r"\1",regex=True)
# dfa.span.str.replace(r" ",r"?",regex=True)
# dfa.span[0]

In [82]:
if str_end > x < y:
    z

NameError: name 'str_end' is not defined

In [83]:
other_idx

NameError: name 'other_idx' is not defined

In [84]:
dfi

,text,start,end,span
0,0: Autophagy maintains tumour growth through c...,0.0,66.0,"(0, 66)"
1,1: Autophagy maintains tumour growth through c...,69.0,135.0,"(69, 135)"


In [95]:
dfa.str_end = dfa.str_end.astype(int)

In [143]:
dfi.reset_index(inplace=True)
dfi.rename(columns={'index':'dfi_idx'},inplace=True)

In [157]:
for o,m in zip(dfi.index,dfi.span):
    x,y = m #open span tuple
    #write conditions for specific df rows
    conds = (dfa.str_end > x) & (dfa.str_end < y)
    dfa.loc[conds,'dfi_idx'] = o #save the index of the dfi span that fits to dfa

In [158]:
dfi

,dfi_idx,text,start,end,span
0,0,0: Autophagy maintains tumour growth through c...,0.0,66.0,"(0, 66)"
1,1,1: Autophagy maintains tumour growth through c...,69.0,135.0,"(69, 135)"


In [159]:
dfa

,id,is_neural_normalized,mention,obj,prob,span,str_end,dfi_idx
0,[mesh:D009369],False,tumour,disease,0.999992,"{'begin': 23, 'end': 29}",29,0
1,[mesh:D001120],False,arginine,drug,0.997611,"{'begin': 57, 'end': 65}",65,0
2,[mesh:D009369],False,tumour,disease,0.999989,"{'begin': 92, 'end': 98}",98,1
3,[mesh:D001120],False,arginine,drug,0.997834,"{'begin': 126, 'end': 134}",134,1


In [160]:
# dfa.merge(dfi, left_on='dfi_idx',right_index=True)
df = dfa.merge(dfi, left_on='dfi_idx',right_on='dfi_idx')

In [162]:
df

,id,is_neural_normalized,mention,obj,prob,span_x,str_end,dfi_idx,text,start,end,span_y
0,[mesh:D009369],False,tumour,disease,0.999992,"{'begin': 23, 'end': 29}",29,0,0: Autophagy maintains tumour growth through c...,0.0,66.0,"(0, 66)"
1,[mesh:D001120],False,arginine,drug,0.997611,"{'begin': 57, 'end': 65}",65,0,0: Autophagy maintains tumour growth through c...,0.0,66.0,"(0, 66)"
2,[mesh:D009369],False,tumour,disease,0.999989,"{'begin': 92, 'end': 98}",98,1,1: Autophagy maintains tumour growth through c...,69.0,135.0,"(69, 135)"
3,[mesh:D001120],False,arginine,drug,0.997834,"{'begin': 126, 'end': 134}",134,1,1: Autophagy maintains tumour growth through c...,69.0,135.0,"(69, 135)"


In [118]:
dfa.columns

Index(['id', 'is_neural_normalized', 'mention', 'obj', 'prob', 'span',
       'str_end'],
      dtype='object')

In [119]:
dfa['dfi_idx'] = ''

In [120]:
dfa.columns

Index(['id', 'is_neural_normalized', 'mention', 'obj', 'prob', 'span',
       'str_end', 'dfi_idx'],
      dtype='object')

In [132]:
dfa.loc[(dfa.str_end < 66)&(dfa.str_end > 0),'dfi_idx'] = '0'
dfa

,id,is_neural_normalized,mention,obj,prob,span,str_end,dfi_idx
0,[mesh:D009369],False,tumour,disease,0.999992,"{'begin': 23, 'end': 29}",29,0
1,[mesh:D001120],False,arginine,drug,0.997611,"{'begin': 57, 'end': 65}",65,0
2,[mesh:D009369],False,tumour,disease,0.999989,"{'begin': 92, 'end': 98}",98,
3,[mesh:D001120],False,arginine,drug,0.997834,"{'begin': 126, 'end': 134}",134,


In [78]:
dfa['dfi_idx'] = ''
for o in dfa.str_end:
    dfi_idx = o
dfa

,id,is_neural_normalized,mention,obj,prob,span,str_end,dfi_idx
0,[mesh:D009369],0.0,tumour,disease,0.999992,"{'begin': 23, 'end': 29}",29,
1,[mesh:D001120],0.0,arginine,drug,0.997611,"{'begin': 57, 'end': 65}",65,
2,[mesh:D009369],0.0,tumour,disease,0.999989,"{'begin': 92, 'end': 98}",98,
3,[mesh:D001120],0.0,arginine,drug,0.997834,"{'begin': 126, 'end': 134}",134,
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65


In [38]:
x

{'annotations': [{'id': ['mesh:D009369'],
   'is_neural_normalized': False,
   'mention': 'tumour',
   'obj': 'disease',
   'prob': 0.9999921321868896,
   'span': {'begin': 23, 'end': 29}},
  {'id': ['mesh:D001120'],
   'is_neural_normalized': False,
   'mention': 'arginine',
   'obj': 'drug',
   'prob': 0.9976105690002441,
   'span': {'begin': 57, 'end': 65}},
  {'id': ['mesh:D009369'],
   'is_neural_normalized': False,
   'mention': 'tumour',
   'obj': 'disease',
   'prob': 0.9999886751174927,
   'span': {'begin': 92, 'end': 98}},
  {'id': ['mesh:D001120'],
   'is_neural_normalized': False,
   'mention': 'arginine',
   'obj': 'drug',
   'prob': 0.9978335499763489,
   'span': {'begin': 126, 'end': 134}}],
 'text': '0: Autophagy maintains tumour growth through circulating arginine :0 1: Autophagy maintains tumour growth through circulating arginine :1',
 'timestamp': 'Thu Nov 10 23:35:56 +0000 2022'}

In [6]:
output = {'annotations': [{'id': ['mesh:D009369'],
   'is_neural_normalized': False,
   'mention': 'tumour',
   'obj': 'disease',
   'prob': 0.9999921321868896,
   'span': {'begin': 23, 'end': 29}},
  {'id': ['mesh:D001120'],
   'is_neural_normalized': False,
   'mention': 'arginine',
   'obj': 'drug',
   'prob': 0.9976105690002441,
   'span': {'begin': 57, 'end': 65}},
  {'id': ['mesh:D009369'],
   'is_neural_normalized': False,
   'mention': 'tumour',
   'obj': 'disease',
   'prob': 0.9999886751174927,
   'span': {'begin': 92, 'end': 98}},
  {'id': ['mesh:D001120'],
   'is_neural_normalized': False,
   'mention': 'arginine',
   'obj': 'drug',
   'prob': 0.9978335499763489,
   'span': {'begin': 126, 'end': 134}}],
 'text': '0: Autophagy maintains tumour growth through circulating arginine :0 1: Autophagy maintains tumour growth through circulating arginine :1',
 'timestamp': 'Thu Nov 10 23:35:56 +0000 2022'}

In [8]:
x = output

In [9]:
output

{'annotations': [{'id': ['mesh:D009369'],
   'is_neural_normalized': False,
   'mention': 'tumour',
   'obj': 'disease',
   'prob': 0.9999921321868896,
   'span': {'begin': 23, 'end': 29}},
  {'id': ['mesh:D001120'],
   'is_neural_normalized': False,
   'mention': 'arginine',
   'obj': 'drug',
   'prob': 0.9976105690002441,
   'span': {'begin': 57, 'end': 65}},
  {'id': ['mesh:D009369'],
   'is_neural_normalized': False,
   'mention': 'tumour',
   'obj': 'disease',
   'prob': 0.9999886751174927,
   'span': {'begin': 92, 'end': 98}},
  {'id': ['mesh:D001120'],
   'is_neural_normalized': False,
   'mention': 'arginine',
   'obj': 'drug',
   'prob': 0.9978335499763489,
   'span': {'begin': 126, 'end': 134}}],
 'text': '0: Autophagy maintains tumour growth through circulating arginine :0 1: Autophagy maintains tumour growth through circulating arginine :1',
 'timestamp': 'Thu Nov 10 23:35:56 +0000 2022'}

In [12]:
dfa = pd.DataFrame(output['annotations'])

In [16]:
output

{'annotations': [{'id': ['mesh:D009369'],
   'is_neural_normalized': False,
   'mention': 'tumour',
   'obj': 'disease',
   'prob': 0.9999921321868896,
   'span': {'begin': 23, 'end': 29}},
  {'id': ['mesh:D001120'],
   'is_neural_normalized': False,
   'mention': 'arginine',
   'obj': 'drug',
   'prob': 0.9976105690002441,
   'span': {'begin': 57, 'end': 65}},
  {'id': ['mesh:D009369'],
   'is_neural_normalized': False,
   'mention': 'tumour',
   'obj': 'disease',
   'prob': 0.9999886751174927,
   'span': {'begin': 92, 'end': 98}},
  {'id': ['mesh:D001120'],
   'is_neural_normalized': False,
   'mention': 'arginine',
   'obj': 'drug',
   'prob': 0.9978335499763489,
   'span': {'begin': 126, 'end': 134}}],
 'text': '0: Autophagy maintains tumour growth through circulating arginine :0 1: Autophagy maintains tumour growth through circulating arginine :1',
 'timestamp': 'Thu Nov 10 23:28:15 +0000 2022'}

In [17]:
df

,id,is_neural_normalized,mention,obj,prob,span
0,[mesh:D009369],False,tumour,disease,0.999992,"{'begin': 23, 'end': 29}"
1,[mesh:D001120],False,arginine,drug,0.997611,"{'begin': 57, 'end': 65}"
2,[mesh:D009369],False,tumour,disease,0.999989,"{'begin': 92, 'end': 98}"
3,[mesh:D001120],False,arginine,drug,0.997834,"{'begin': 126, 'end': 134}"
